# Traditional methods

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

# Load the datasets
train_data = pd.read_csv('./goemotions/train.tsv', sep='\t', header=None)
val_data = pd.read_csv('./goemotions/dev.tsv', sep='\t', header=None)
test_data = pd.read_csv('./goemotions/test.tsv', sep='\t', header=None)

train_data.columns = ['text', 'label', 'code']
val_data.columns = ['text', 'label', 'code']
test_data.columns = ['text', 'label', 'code']

# Basic preprocessing
def preprocess_text(df):
    df['cleaned_text'] = df['text'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
    return df

train_data = preprocess_text(train_data)
val_data = preprocess_text(val_data)
test_data = preprocess_text(test_data)

# Split the 'label' column into a list of labels (comma-separated integers)
def split_labels(label):
    # Convert the label string (e.g., "6,22") into a list of integers [6, 22]
    return list(map(int, label.split(',')))

train_data['label'] = train_data['label'].apply(split_labels)
val_data['label'] = val_data['label'].apply(split_labels)
test_data['label'] = test_data['label'].apply(split_labels)

# Use MultiLabelBinarizer to encode the labels into binary format
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_data['label'])
y_test = mlb.fit_transform(test_data['label'])
y_val = mlb.transform(val_data['label'])

X_train = train_data['cleaned_text']
X_test = test_data['cleaned_text']
X_val = val_data['cleaned_text']

# Vectorize the text data using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)

# Define classifiers with OneVsRestClassifier for multilabel classification
classifiers = {
    'Logistic Regression': OneVsRestClassifier(LogisticRegression(max_iter=1000)),
    'Support Vector Classifier': OneVsRestClassifier(SVC(probability=True)),
    'Multinomial Naive Bayes': MultinomialNB(),  # No need for OneVsRest
    'Random Forest': OneVsRestClassifier(RandomForestClassifier())
}

# # Train and evaluate each classifier on the validation set
# for name, classifier in classifiers.items():
#     # Create pipeline with TF-IDF and classifier
#     model = Pipeline([
#         ('tfidf', tfidf),
#         ('clf', classifier)
#     ])
    
#     # Train the model
#     model.fit(X_train, y_train)
    
#     # Predict on validation data
#     if name == 'Support Vector Classifier':
#         y_val_pred_proba = model.decision_function(X_val)
#         y_val_pred = np.where(y_val_pred_proba > 0, 1, 0)  
#     else:
#         # For other classifiers, use predict directly
#         y_val_pred = model.predict(X_val)
    
#     # Evaluate the model on validation data
#     print(f"Results for {name}:")
#     print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
#     print("Classification Report:")
#     print(classification_report(y_val, y_val_pred, target_names=mlb.classes_))
#     print("\n" + "="*60 + "\n")


# Deep learning

## tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report

# Tokenization and Padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

maxlen = 100
epochs_num = 2
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_val_pad = pad_sequences(X_val_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

# Define the LSTM model for multilabel classification
lstm_model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=maxlen),
    LSTM(units=128, dropout=0.2, recurrent_dropout=0.2),
    Dense(units=len(mlb.classes_), activation='sigmoid')  # Use sigmoid for multilabel classification
])

# Compile the model using binary_crossentropy loss for multilabel classification
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
lstm_model.fit(X_train_pad, y_train, epochs=epochs_num, batch_size=64, validation_data=(X_val_pad, y_val))

# Predict on validation data
lstm_val_pred = lstm_model.predict(X_val_pad)

# Convert probabilities to binary labels (using a threshold of 0.5)
lstm_val_pred_classes = (lstm_val_pred > 0.5).astype(int)

# Evaluate the model using classification_report
print("LSTM:\n", classification_report(y_val, lstm_val_pred_classes, target_names=mlb.classes_))


In [ ]:
import tensorflow as tf


# List physical devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Available physical devices:", physical_devices)

# Check if TensorFlow is using the GPU
print("Is TensorFlow using GPU?", tf.test.is_gpu_available())


## transformers

In [ ]:
import tensorflow as tf
import transformers
from transformers import TFBertForSequenceClassification, BertTokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC

# Number of labels (using len(mlb.classes_) from MultiLabelBinarizer)
num_labels = len(mlb.classes_)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Tokenization function
def tokenize_data(data, max_length=128):
    return tokenizer(
        data['text'].tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

# Tokenize datasets
X_train = tokenize_data(train_data)
X_val = tokenize_data(val_data)
X_test = tokenize_data(test_data)
batch_size = 32

# Prepare TensorFlow dataset objects for multilabel classification
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': X_train['input_ids'], 
        'attention_mask': X_train['attention_mask'], 
        'token_type_ids': X_train['token_type_ids']
    },
    y_train  # Use multilabel binarized labels here (already binarized using MultiLabelBinarizer)
)).batch(batch_size).shuffle(1000)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': X_val['input_ids'], 
        'attention_mask': X_val['attention_mask'], 
        'token_type_ids': X_val['token_type_ids']
    },
    y_val  # Use multilabel binarized labels here
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': X_test['input_ids'], 
        'attention_mask': X_test['attention_mask'], 
        'token_type_ids': X_test['token_type_ids']
    },
    y_test  # Use multilabel binarized labels here
)).batch(batch_size)

# Compile the model for multilabel classification
model.compile(
    optimizer=Adam(learning_rate=2e-5),
    loss=BinaryCrossentropy(from_logits=True),  # Use binary cross-entropy for multilabel classification
    metrics=[AUC(name='auc')]  # Use AUC as a metric for multilabel classification
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

# Evaluate the model on the test set
test_loss, test_auc = model.evaluate(test_dataset)
print(f'Test AUC: {test_auc}')


# Evaluation and comparison

In [ ]:
# Assuming we have collected the classification reports
results = {
    'Logistic Regression': lr_val_pred,
    'SVM': svm_val_pred,
    'Naive Bayes': nb_val_pred,
    'Random Forest': rf_val_pred,
    'LSTM': lstm_val_pred_classes,
    # 'BERT': val_pred_classes
    # Add GRU, CNN, and BERT results
}

# Compare accuracy, precision, recall, and F1-score
for model, preds in results.items():
    print(f"{model}:\n", classification_report(y_val, preds))
    
metrics = {}

for model, preds in results.items():
    report = classification_report(y_val, preds, output_dict=True)
    metrics[model] = {
        'accuracy': report['accuracy'],
        'macro avg': {
            'precision': report['macro avg']['precision'],
            'recall': report['macro avg']['recall'],
            'f1-score': report['macro avg']['f1-score']
        },
        'weighted avg': {
            'precision': report['weighted avg']['precision'],
            'recall': report['weighted avg']['recall'],
            'f1-score': report['weighted avg']['f1-score']
        }
    }

# Print collected metrics
for model, stats in metrics.items():
    print(f"{model}:")
    print(f"  Accuracy: {stats['accuracy']:.4f}")
    print(f"  Macro Average Precision: {stats['macro avg']['precision']:.4f}")
    print(f"  Macro Average Recall: {stats['macro avg']['recall']:.4f}")
    print(f"  Macro Average F1-score: {stats['macro avg']['f1-score']:.4f}")
    print(f"  Weighted Average Precision: {stats['weighted avg']['precision']:.4f}")
    print(f"  Weighted Average Recall: {stats['weighted avg']['recall']:.4f}")
    print(f"  Weighted Average F1-score: {stats['weighted avg']['f1-score']:.4f}")


# Evaluation on test set

In [ ]:
# Evaluate the best model on the test set
best_model = lstm_model  # Assuming LSTM is the best model
test_pred = best_model.predict(X_test_pad)
test_pred_classes = np.argmax(test_pred, axis=1)
print("Final Model Evaluation on Test Set:\n", classification_report(y_test, test_pred_classes))


# Visualization

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# List of models and their predicted labels
models_predictions = {
    'Logistic Regression': lr_val_pred,
    'SVM': svm_val_pred,
    'Naive Bayes': nb_val_pred,
    'Random Forest': rf_val_pred
}

# Function to extract metrics
def extract_metrics(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)
    accuracy = report['accuracy']
    precision = np.mean([report[label]['precision'] for label in report if label.isdigit()])
    recall = np.mean([report[label]['recall'] for label in report if label.isdigit()])
    f1_score = np.mean([report[label]['f1-score'] for label in report if label.isdigit()])
    return accuracy, precision, recall, f1_score

# Initialize lists for metrics
accuracies, precisions, recalls, f1_scores = [], [], [], []

# Calculate metrics for each model
for model_name, y_pred in models_predictions.items():
    accuracy, precision, recall, f1_score = extract_metrics(y_test, y_pred)
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)
    print(f"{model_name} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1_score}")
    
# Plotting the metrics
def plot_metric(models, metric_values, metric_name):
    plt.figure(figsize=(10, 6))
    sns.barplot(x=models, y=metric_values)
    plt.title(f'{metric_name} Comparison')
    plt.xlabel('Models')
    plt.ylabel(metric_name)
    plt.ylim(0, 1)
    plt.show()

models = list(models_predictions.keys())

plot_metric(models, accuracies, 'Accuracy')
plot_metric(models, precisions, 'Precision')
plot_metric(models, recalls, 'Recall')
plot_metric(models, f1_scores, 'F1 Score')

# Plot confusion matrix for each model
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title(f'Confusion Matrix for {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

for model_name, y_pred in models_predictions.items():
    plot_confusion_matrix(y_test, y_pred, model_name)
    
# Assuming 'y_train' and 'y_test' are encoded labels
plt.figure(figsize=(8, 6))
sns.countplot(x=y_train)
plt.title('Class Distribution in Training Set')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Załóżmy, że y_true to rzeczywiste etykiety, a y_pred to przewidywane etykiety
# y_true = ...  # rzeczywiste etykiety
# y_pred = ...  # przewidywane etykiety

# Generowanie macierzy pomyłek
cm = confusion_matrix(y_test, lstm_val_pred_classes)

# Wizualizacja macierzy pomyłek
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Zakładamy, że masz historyczny obiekt z trenowania modelu: history = model.fit(...)

# Rysowanie wykresu dokładności
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(loc='lower right')
plt.show()

# Rysowanie wykresu strat
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend(loc='upper right')
plt.show()